In [1]:
!pip install nltk
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095401 sha256=ec3476de8ea780349ea070e4eb1d3146e8585ad5e8b1cb58b91ad3095fc779bc
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

from surprise import Dataset
from surprise import Reader
from surprise import SVD

import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

In [3]:
movies_df = pd.read_csv('/content/movies_metadata.csv')
movies_df.head()

<ipython-input-3-185d13d355d8>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv('/content/movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [5]:
movies_df_clean = movies_df[['adult','genres','id','original_language','overview','popularity','production_companies','production_countries','tagline','title','vote_average','vote_count']]

In [6]:
# Convert the 'id' column to int64 data type and replace missing values with a unique identifier
movies_df_clean['id'] = pd.to_numeric(movies_df_clean['id'], errors='coerce').astype('Int64')
movies_df_clean['id'].fillna(-1, inplace=True)

# Drop rows with missing 'id' values
movies_df_clean = movies_df_clean.dropna(subset=['id'])

<ipython-input-6-92d7efc20556>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df_clean['id'] = pd.to_numeric(movies_df_clean['id'], errors='coerce').astype('Int64')
<ipython-input-6-92d7efc20556>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df_clean['id'].fillna(-1, inplace=True)


In [8]:
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
def preprocess_title(title):
    if isinstance(title, str):
        tokens = nltk.word_tokenize(title.lower())
        lemmatized_tokens = [lemmatizer.lemmatize(token, wordnet.VERB) for token in tokens]
        return ' '.join(lemmatized_tokens)
    else:
        return ''

In [11]:
movies_df_clean['title'] = movies_df_clean['title'].fillna('')  # Replace NaN values with empty string
movies_df_clean['processed_title'] = movies_df_clean['title'].apply(preprocess_title)

In [12]:
movies_df_clean.head()

,adult,genres,id,original_language,overview,popularity,production_companies,production_countries,tagline,title,vote_average,vote_count,processed_title
0,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",NaN,Toy Story,7.7,5415.0,toy story
1,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,jumanji
2,False,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,grumpier old men
3,False,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,wait to exhale
4,False,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,father of the bride part ii


In [15]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    if isinstance(text, str):
        # Tokenize the text
        tokens = word_tokenize(text.lower())
        
        # Remove stopwords and punctuation
        tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]
        
        # Perform lemmatization
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
        
        # Join the tokens back into a single string
        processed_text = ' '.join(lemmatized_tokens)
        
        return processed_text
    else:
        return ''


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
from sklearn.cluster import KMeans

# Preprocess the 'overview' column
movies_df_clean['processed_overview'] = movies_df_clean['overview'].apply(preprocess_text)

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(movies_df_clean['processed_overview'])

# Perform K-means clustering
num_clusters = 5  # Number of clusters to create
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

# Add the cluster labels to the dataframe
movies_df_clean['cluster_label'] = kmeans.labels_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [18]:
movies_df_clean.head()

,adult,genres,id,original_language,overview,popularity,production_companies,production_countries,tagline,title,vote_average,vote_count,processed_title,processed_overview,cluster_label
0,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",NaN,Toy Story,7.7,5415.0,toy story,led woody andy 's toy live happily room andy '...,4
1,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,jumanji,sibling judy peter discover enchanted board ga...,4
2,False,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,grumpier old men,family wedding reignites ancient feud next-doo...,4
3,False,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,wait to exhale,cheated mistreated stepped woman holding breat...,3
4,False,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,father of the bride part ii,george bank recovered daughter 's wedding rece...,0


In [20]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_genres(genres):
    if isinstance(genres, list):
        # Extract the genre names from the list of dictionaries
        genre_names = [genre['name'] for genre in genres]
        
        # Join the genre names into a single string
        genres_text = ' '.join(genre_names)
        
        # Tokenize the text
        tokens = word_tokenize(genres_text.lower())
        
        # Remove stopwords and punctuation
        tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]
        
        # Perform lemmatization
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
        
        # Join the tokens back into a single string
        processed_genres = ' '.join(lemmatized_tokens)
        
        if processed_genres == '':
            processed_genres = 'other'  # Assign a default label for empty genres
        
        return processed_genres
    else:
        return 'other'

# Preprocess the 'genres' column
movies_df_clean['processed_genres'] = movies_df_clean['genres'].apply(preprocess_genres)

# Create a TF-IDF vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

# Fit and transform the processed genres column
genres_vectors = vectorizer.fit_transform(movies_df_clean['processed_genres'])

# Perform clustering
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, random_state=42)
clusters = kmeans.fit_predict(genres_vectors)

# Add the cluster labels to the dataframe
movies_df_clean['cluster'] = clusters


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1033: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  return self.fit(X, sample_weight=sample_weight).labels_


In [21]:
movies_df_clean.head()

,adult,genres,id,original_language,overview,popularity,production_companies,production_countries,tagline,title,vote_average,vote_count,processed_title,processed_overview,cluster_label,processed_genres,cluster
0,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",NaN,Toy Story,7.7,5415.0,toy story,led woody andy 's toy live happily room andy '...,4,other,0
1,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,jumanji,sibling judy peter discover enchanted board ga...,4,other,0
2,False,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,grumpier old men,family wedding reignites ancient feud next-doo...,4,other,0
3,False,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,wait to exhale,cheated mistreated stepped woman holding breat...,3,other,0
4,False,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,father of the bride part ii,george bank recovered daughter 's wedding rece...,0,other,0


In [22]:
movies_df_clean.columns

Index(['adult', 'genres', 'id', 'original_language', 'overview', 'popularity',
       'production_companies', 'production_countries', 'tagline', 'title',
       'vote_average', 'vote_count', 'processed_title', 'processed_overview',
       'cluster_label', 'processed_genres', 'cluster'],
      dtype='object')

In [23]:
ratings_df = pd.read_csv('/content/ratings.csv')
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [24]:
ratings_df_clean = ratings_df[['movieId','rating']].groupby(by='movieId').mean()

In [25]:
ratings_df_clean.reset_index(inplace=True)

In [26]:
combined_df_clean = pd.merge(ratings_df_clean, movies_df_clean, left_on='movieId', right_on='id')
combined_df_clean

,movieId,rating,adult,genres,id,original_language,overview,popularity,production_companies,production_countries,tagline,title,vote_average,vote_count,processed_title,processed_overview,cluster_label,processed_genres,cluster
0,2,3.236953,False,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",2,fi,Taisto Kasurinen is a Finnish coal miner whose...,3.860491,"[{'name': 'Villealfa Filmproduction Oy', 'id':...","[{'iso_3166_1': 'FI', 'name': 'Finland'}]",NaN,Ariel,7.1,44.0,ariel,taisto kasurinen finnish coal miner whose fath...,0,other,0
1,3,3.175550,False,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",3,fi,"An episode in the life of Nikander, a garbage ...",2.29211,"[{'name': 'Villealfa Filmproduction Oy', 'id':...","[{'iso_3166_1': 'FI', 'name': 'Finland'}]",NaN,Shadows in Paradise,7.1,35.0,shadow in paradise,episode life nikander garbage man involving de...,0,other,0
2,5,3.079565,False,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",5,en,It's Ted the Bellhop's first night on the job....,9.026586,"[{'name': 'Miramax Films', 'id': 14}, {'name':...","[{'iso_3166_1': 'US', 'name': 'United States o...",Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,539.0,four room,'s ted bellhop 's first night job ... hotel 's...,4,other,0
3,6,3.841764,False,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",6,en,"While racing to a boxing match, Frank, Mike, J...",5.538671,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",Don't move. Don't whisper. Don't even breathe.,Judgment Night,6.4,79.0,judgment night,racing boxing match frank mike john rey get ba...,4,other,0
4,11,3.660591,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",11,en,Princess Leia is captured and held hostage by ...,42.149697,"[{'name': 'Lucasfilm', 'id': 1}, {'name': 'Twe...","[{'iso_3166_1': 'US', 'name': 'United States o...","A long time ago in a galaxy far, far away...",Star Wars,8.1,6778.0,star war,princess leia captured held hostage evil imper...,4,other,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7564,176077,3.500000,False,"[{'id': 53, 'name': 'Thriller'}]",176077,en,A psychological thriller set in the world of u...,1.219246,"[{'name': 'Cimarron Productions', 'id': 4554},...","[{'iso_3166_1': 'US', 'name': 'United States o...",NaN,Enter the Dangerous Mind,4.8,14.0,enter the dangerous mind,psychological thriller set world underground d...,4,other,0
7565,176085,3.000000,False,"[{'id': 35, 'name': 'Comedy'}]",176085,en,"After an unexpected tragedy, Suzanne searches ...",0.787464,"[{'name': 'IFC Films', 'id': 307}]","[{'iso_3166_1': 'US', 'name': 'United States o...",NaN,White Reindeer,5.3,10.0,white reindeer,unexpected tragedy suzanne search true meaning...,4,other,0
7566,176143,3.500000,False,"[{'id': 10752, 'name': 'War'}, {'id': 18, 'nam...",176143,en,A Japanese publisher urges his American-educat...,0.700467,"[{'name': 'RKO Radio Pictures', 'id': 6}]","[{'iso_3166_1': 'US', 'name': 'United States o...",The year's most timely story.,Behind the Rising Sun,6.0,2.0,behind the rise sun,japanese publisher urge american-educated son ...,4,other,0
7567,176167,3.000000,False,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",176167,de,Hans-Jürgen Syberberg's Parsifal film is a psy...,0.165026,"[{'name': 'Gaumont', 'id': 9}, {'name': 'Bavar...","[{'iso_3166_1': 'DE', 'name': 'Germany'}]",NaN,Parsifal,8.3,3.0,parsifal,hans-jürgen syberberg 's parsifal film psychol...,1,other,0


In [33]:
# Create a TF-IDF vectorizer for overview
overview_vectorizer = TfidfVectorizer()
overview_vectors = overview_vectorizer.fit_transform(combined_df_clean['processed_overview'])

In [34]:
# Create a TF-IDF vectorizer for genres
genres_vectorizer = TfidfVectorizer()
genres_vectors = genres_vectorizer.fit_transform(combined_df_clean['processed_genres'])

# Compute the similarity scores for overview
overview_similarity_scores = cosine_similarity(overview_vectors)

# Compute the similarity scores for genres
genres_similarity_scores = cosine_similarity(genres_vectors)

In [38]:
def find_movie_id(movie_title):
    try:
        movie_id = combined_df_clean[combined_df_clean['processed_title'] == movie_title]['id'].values[0]
        return movie_id
    except IndexError:
        return None

def find_similar_movies(movie_title, top_n=5):
    movie_id = find_movie_id(movie_title)
    
    if movie_id is None:
        print(f"Sorry, I can't recognize the movie '{movie_title}'.")
        return []
    
    # Get the similarity scores for the movie
    movie_index = combined_df_clean[combined_df_clean['id'] == movie_id].index[0]
    overview_similarity_scores_movie = overview_similarity_scores[movie_index]
    genres_similarity_scores_movie = genres_similarity_scores[movie_index]
    
    # Combine the similarity scores
    combined_similarity_scores = 0.5 * overview_similarity_scores_movie + 0.5 * genres_similarity_scores_movie
    
    # Find the indices of similar movies based on the combined similarity scores
    similar_movie_indices = combined_similarity_scores.argsort()[::-1][1:top_n+1]
    
    # Get the titles of similar movies
    similar_movies = combined_df_clean.loc[similar_movie_indices, 'title'].tolist()
    
    return similar_movies

In [39]:
while True:
    favorite_movie = input("Enter your favorite movie (or 'exit' to quit): ")
    
    if favorite_movie.lower() == 'exit':
        break
    
    similar_movies = find_similar_movies(favorite_movie)
    
    if similar_movies:
        print(f"Similar movies based on '{favorite_movie}':")
        for movie_title in similar_movies:
            print(movie_title)
    else:
        print(f"Sorry, no similar movies found for '{favorite_movie}'.")

Enter your favorite movie (or 'exit' to quit): toy stoy
Sorry, I can't recognize the movie 'toy stoy'.
Sorry, no similar movies found for 'toy stoy'.
Enter your favorite movie (or 'exit' to quit): toy story
Similar movies based on 'toy story':
The 40 Year Old Virgin
Wabash Avenue
Man on the Moon
Malice
Judge Hardy and Son
Enter your favorite movie (or 'exit' to quit): inception
Similar movies based on 'inception':
Primeval
One in the Chamber
The Last Seduction II
Bloodline
The Jackal
Enter your favorite movie (or 'exit' to quit): the godfather
Similar movies based on 'the godfather':
The Godfather: Part II
The Godfather: Part III
Household Saints
Violent City
Gang War in Milan
Enter your favorite movie (or 'exit' to quit): lucy
Similar movies based on 'lucy':
My Best Fiend
Paris is Burning
Best Foot Forward
Critic's Choice
The Negro Soldier
Enter your favorite movie (or 'exit' to quit): tom & jerry
Sorry, I can't recognize the movie 'tom & jerry'.
Sorry, no similar movies found for 'to